In [44]:
# Import 
import numpy as np # linear algebra
import pandas as pd # data processing

import os
for dirname, _, filenames in os.walk('tiny-imagenet-200/'):
    for filename in filenames:
        os.path.join(dirname, filename)



In [45]:
# Import of all libraries needed in the project

from tensorflow import keras
from keras.layers import Input, Dense
from tensorflow.keras.layers import Input, Dense
from keras import applications
from keras.layers import Activation,Add, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D        
from tensorflow.keras.layers import Activation,Add, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D  
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model,Model
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.initializers import *
from keras import regularizers
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [46]:
import cv2
from tqdm import tqdm

train=os.listdir('tiny-imagenet-200/train') # Load Dataset
X_train=[]
y_train=[]
for folderName in train:
    for image_filename in tqdm(os.listdir('tiny-imagenet-200/train/' + folderName +'/'+'images')):
        
                img_file = cv2.imread('tiny-imagenet-200/train/'+ folderName + '/' +'images' + '/'+image_filename)

                img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2RGB)
                img_arr = np.asarray(img_file)

                
                X_train.append(img_arr)
                y_train.append(folderName)
print(np.array(X_train).shape)
print(np.array(y_train).shape) #printing shape of data

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2493.75it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2469.79it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2124.00it/s]


(100000, 64, 64, 3)
(100000,)


In [47]:

val_dataframe=pd.read_csv('tiny-imagenet-200/val/val_annotations.txt',sep='\t',names=['id','label','d1','d2','d3','d4'])
val_dataframe=val_dataframe.iloc[0:,0:2]
val_dataframe.sort_values(["id"], axis=0, ascending=True, inplace=True)

X_val=[]
y_val=[]
for image_filenames in tqdm(os.listdir("tiny-imagenet-200/val/images")):
    img_file = cv2.imread('tiny-imagenet-200/val/images/' + image_filenames) 
    img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2RGB)
    img_arr = np.asarray(img_file)
    #img_arr=img_arr/255
    X_val.append(img_arr)
    
    for i in range (len(val_dataframe['id'].values)):
        if(image_filenames==val_dataframe['id'].values[i]):
             y_val.append(val_dataframe['label'].values[i])

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [06:33<00:00, 25.44it/s]


In [48]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


y_train= LabelEncoder().fit_transform(y_train) # transform and fitting data 
y_train_hot=keras.utils.to_categorical(y_train, num_classes=200) # conversion to one hot vector


y_val=LabelEncoder().fit_transform(y_val)
y_val_hot=keras.utils.to_categorical(y_val, num_classes=200)
y_val_hot.shape

(10000, 200)

In [49]:
X_val=np.array(X_val)
X_train=np.array(X_train)
X_train.shape

(100000, 64, 64, 3)

In [50]:
datagen_train = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_generator = datagen_train.flow_from_directory('tiny-imagenet-200/train', target_size = (64,64), batch_size= 32, class_mode="categorical")


datagen_val=ImageDataGenerator(rescale=1./255)
val_generator=datagen_val.flow(X_val,y_val_hot,batch_size=32)

Found 100000 images belonging to 200 classes.


In [51]:
img_height,img_width = 64,64 
num_classes = 200

base_model =keras.applications.Xception(include_top=False, weights='imagenet', classes=200)

In [52]:
from keras import regularizers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.8)(x)
predictions = Dense(num_classes, activation= 'softmax', kernel_regularizer=regularizers.l2(0.001))(x)
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()

# Model Design and adding layers 
# It was showing error Node has no attribute OrderMask and so I imported the layers from tensorflow.keras rather than kers

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [53]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer= SGD(lr=0.001, momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

# Compiling Model

In [54]:
model.save_weights("best_model_resnet50v2_2.h5")
checkpoint = ModelCheckpoint("best_model_resnet50v2_2.h5",monitor='val_accuracy',verbose=1,
                              save_best_only=True,mode='max')
callback=[checkpoint] 

In [55]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = 90000//100,
                              epochs = 100,
                              validation_data = val_generator,
                              validation_steps = 10000//100,callbacks=callback)

# Training Model 

Epoch 1/100
 324/3125 [==>...........................] - ETA: 4:40:07 - loss: 5.6905 - acc: 0.0053

KeyboardInterrupt: 

### Interrupted due to time consumption

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 100, 1))
#ax1.set_yticks(np.arange(0, 10, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 100, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

# Plots



In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

data_test=[]
file=glob('tiny-imagenet-200/test/images/*.JPEG')
for f in file:
    images=load_img(f)
    image_array=img_to_array(images)
    image_array=image_array/255
    data_test.append(image_array)
    images=None
    image_array=None
x_test=np.array(data_test)
x_test.shape

# Loading Testing Data

In [ ]:
prediction = model.predict(x_test)
prediction

# Getting Predictions for our Testing Data

In [ ]:
predicted_class_indices=np.argmax(prediction,axis=1)
predicted_class_indices

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in predicted_class_indices]
test_filenames = os.listdir("tiny-imagenet-200/test/images")
test_df = pd.DataFrame({'file_name': test_filenames,"category":pred})
test_df.sort_values(["file_name"], axis=0, ascending=True, inplace=True)   



#test_df.to_csv('submission.csv',header=True, index=False) 


# Can uncomment the above line to save the outputs.

### Due to high time consuming I stopped the training part... But Yes you can train and test on your machine or on Google Colab

## Done 
## By Hamdan